## (함수) csv 파일을 데이터프레임으로 불러오는 함수

In [1]:
import pandas as pd 
def openDataframe(fileName):
    return pd.read_csv(fileName)

In [2]:
import pandas as pd
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500 = sp500[['Symbol', 'Security']]
sp500 = sp500.rename(columns={'Symbol': 'Ticker', 'Security': 'Company'})
sp500

,Ticker,Company
0,MMM,3M
1,AOS,A. O. Smith
2,ABT,Abbott
3,ABBV,AbbVie
4,ACN,Accenture
...,...,...
498,YUM,Yum! Brands
499,ZBRA,Zebra Technologies
500,ZBH,Zimmer Biomet
501,ZION,Zions Bancorporation


# (함수) 기능 설명: 입력->출력 

## (함수) 개별 기업의 데이터셋을 불러오는 함수: 기업 티커심볼->데이터프레임

In [95]:
def openCorpDataframe(ticker):
    folder_path = './SP500-weekly-price/'
    file_name = f'weekly_{ticker}.csv'
    df = pd.read_csv(folder_path + file_name)
    df.reset_index(drop=True, inplace=True)

    return df

## (함수) 주가 정보의 각 날짜에 해당하는 S&P index를 추가하는 함수: 데이터프레임->데이터프레임

In [96]:
import numpy as np

def getDataframe_withSPX(df_ticker):
    df = df_ticker.copy()
    df_SP = pd.read_csv('dataset-SP500index-daily.csv')
    
    bucket_SPX = []
    for i in range(len(df)):
        if df_SP['SP500'][df_SP['DATE']==df['timestamp'].loc[i]].empty:
            bucket_SPX.append(np.NaN)        
        else:
            bucket_SPX.append(float(df_SP['SP500'][df_SP['DATE']==df['timestamp'].loc[i]]))
    df['SPX'] = bucket_SPX
    df.dropna(inplace=True)

    return df

## (함수) 주가 정보에 가격수익률과 인덱스 수익률을 추가하는 함수: 데이터프레임->데이터프레임

In [160]:
def getDataframe_withReturns(df_ticker):
    df = df_ticker.copy()
    df = df.iloc[::-1]
    df.reset_index(drop=True, inplace=True)
    df['return_price'] = round(df['close'].pct_change(), 7)
    df['return_SPX'] = round(df['SPX'].pct_change(), 7)
    df.dropna(inplace=True)
    df = df[df['return_price'] > -0.9] # 예외처리(주식분할 후 수정주가 미반영에 의한 하락 삭제)
    df = df[df['timestamp'] > '2018-01-01'] # 최근 5년 주가 정보에 대해 분석 수행
    df.reset_index(drop=True, inplace=True)

    return df

## (함수) 주가 정보에 정규화된 가격과 정규화된 인덱스를 추가하는 함수: 데이터프레임->데이터프레임

In [183]:
def getDataframe_withNormed(df_ticker):
    df = df_ticker.copy()
    df.reset_index(drop=True, inplace=True)
    df['normed_price'] = df['close']/df.loc[0]['close']
    df['normed_SPX'] = df['SPX']/df.loc[0]['SPX']

    return df

## (함수) : 주가 정보 데이터셋으로부터 주가 캔들차트, 거래량 막대차트를 출력하는 함수: 데이터프레임->플롯

- 실행을 위해 plotly 라이브러리 설치가 필요합니다. (아래 셀)

In [377]:
# !pip install plotly

In [368]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

def showCandleStickChart(df_ticker, ticker='company'):
    # 데이터셋
    df = df_ticker.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    # 서브플롯 생성
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # 캔들스틱 차트 생성
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='Price'), row=1, col=1)

    # 거래량 바차트 생성
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=2, col=1)

    fig.update_layout(
        title=f'{ticker}: Stock Price Chart with Volume (week)',
        yaxis=dict(title='Price', domain=[0.4, 1]),
        xaxis=dict(rangeslider=dict(visible=False)),
        yaxis2=dict(title='Volume', domain=[0, 0.35]),
        xaxis2=dict(rangeslider=dict(visible=True)),  # 스크롤 기능 활성화
        height=600
    )

    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"{ticker}-price-vol_{current_time}.png"
    fig.write_image(filename)   
    fig.show()
    
    

## (함수) : 주가 정보 데이터셋으로부터 주가와 인덱스 변화를 비교 시각화하는 함수: 데이터프레임->플롯

In [369]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

def showPriceSPXPlot(df_ticker, ticker='company'):
    # 주어진 데이터프레임 생성
    df = df_ticker

    # 서브플롯 생성
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Trace 생성
    trace_price = go.Scatter(x=df['timestamp'], y=df['normed_price'], name='normed_price')
    trace_spx = go.Scatter(x=df['timestamp'], y=df['normed_SPX'], name='normed_SPX')

    # 데이터 추가
    fig.add_trace(trace_price, secondary_y=False)
    fig.add_trace(trace_spx, secondary_y=False)

    # 축 설정
    fig.update_xaxes(title_text='timestamp')
    fig.update_yaxes(title_text='Normalized value', secondary_y=False)

    # 레이아웃 설정
    fig.update_layout(
        title=f'{ticker}: Price vs SPX',
        legend=dict(
            x=0.01,
            y=0.99,
            traceorder='normal',
            font=dict(family='sans-serif', size=12, color='black')
        )
    )
    
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"{ticker}-vsSPX_{current_time}.png"
    fig.write_image(filename)   

    # 그래프 출력
    fig.show()


## (함수) 데이터로부터 CAPM beta를 analytic하게 구하기: 데이터프레임->계산결과

- CAPM 계산에 필요한 risk free asset의 수익률은 미 국채1년을 기본 설정으로 함 (수익률 4.79%, 변수 입력으로 설정 변경 가능)
- CAPM beta: $ \beta_i = \frac{Cov(r_i, r_M)}{Var(r_M)}$

In [383]:
def getCAPMvalues(df_ticker, ticker='company', r_f=4.79/100):
    covariance = df_ticker['return_price'].cov(df_ticker['return_SPX'])
    variance = df_ticker['return_SPX'].var()
    expectation = df_ticker['return_SPX'].mean()
    beta = covariance/variance
    r_M = expectation
    r = r_f + beta*(r_M - r_f)
    return {'ticker': ticker, 'beta': beta, 'r_CAPM' : r, 'cov': covariance, 'r_M': r_M, 'r_f': r_f}

## (함수) 데이터로부터 CAPM beta를 선형회귀로 추정하기: 데이터프레임->플롯, 계산결과

In [370]:
import plotly.graph_objects as go
import statsmodels.api as sm
from datetime import datetime

def showLinREgCAPMPlot(df_ticker, ticker='company', r_f = 0.0479):
    # 데이터프레임 설정
    df = df_ticker

    # 회귀 분석
    y = df['return_price']-r_f
    X = df['return_SPX']-r_f
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()

    # 산점도 그리기
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['return_SPX'],
        y=df['return_price'],
        mode='markers',
        name='data'
    ))

    # 회귀선 그리기
    fig.add_trace(go.Scatter(
        x=df['return_SPX'],
        y=results.fittedvalues,
        mode='lines',
        line=dict(color='red'),
        name='lin.reg. line'
    ))

    # 축 레이블 설정
    fig.update_layout(
        title=f'{ticker}: CAPM by Linear Regression',
        xaxis_title='Return_SPX',
        yaxis_title='Return_Price'
    )

    # 회귀식 텍스트 추가
    fig.add_annotation(
        x=0.01,
        y=0.03,
        text=f"r_Price - r_f = {results.params['return_SPX']:.4f} * (r_SPX - r_f) + {results.params['const']:.4f}",
        showarrow=False,
        font=dict(size=12),
        align='left',
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='middle'
    )

    # 그래프 출력
    fig.show()
    
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"{ticker}-linreg_{current_time}.png"
    fig.write_image(filename)   
    
    print(f'{ticker}: CAPM beta = {results.params["return_SPX"]:.4f}')


# (함수) 데이터셋 변환, 계산 등 모든 기능을 한번에 실행해주는 함수: 티커심볼->데이터프레임과 계산결과

In [384]:
def operateCAPM(ticker):
    df_ticker = openCorpDataframe(ticker)
    df_ticker = getDataframe_withSPX(df_ticker)
    df_ticker = getDataframe_withReturns(df_ticker)
    df_ticker = getDataframe_withNormed(df_ticker)
    dict_ticker = getCAPMvalues(df_ticker, ticker)
    return {'df': df_ticker, 'capm': dict_ticker}

# (실행 예시) CAPM: V(비자)

In [385]:
# 티커심볼을 입력한 단 한줄로 CAPM 분석 실행
capm_V = operateCAPM('V')

In [386]:
# 위 출력에 ['df']로 데이터프레임 조회 가능
df_V = capm_V['df']
df_V

,timestamp,open,high,low,close,volume,SPX,return_price,return_SPX,normed_price,normed_SPX
0,2018-01-05,114.57,119.000,113.95,118.86,28091199,2743.15,0.042449,0.026010,1.000000,1.000000
1,2018-01-12,118.61,120.480,117.46,120.09,31608196,2786.24,0.010348,0.015708,1.010348,1.015708
2,2018-01-19,121.18,123.650,119.91,122.70,32530633,2810.30,0.021734,0.008635,1.032307,1.024479
3,2018-01-26,122.87,126.380,122.40,126.32,31242833,2872.87,0.029503,0.022264,1.062763,1.047289
4,2018-02-02,126.74,126.880,120.70,120.91,45770049,2762.13,-0.042828,-0.038547,1.017247,1.006919
...,...,...,...,...,...,...,...,...,...,...,...
273,2023-03-31,222.59,225.840,218.65,225.46,33005510,4109.31,0.019996,0.034833,1.896853,1.498026
274,2023-04-06,225.23,230.050,224.12,225.99,20775193,4105.02,0.002351,-0.001044,1.901312,1.496462
275,2023-04-14,225.56,234.990,224.32,234.02,31235537,4137.64,0.035533,0.007946,1.968871,1.508354
276,2023-04-21,234.02,235.568,230.94,234.05,24375871,4133.52,0.000128,-0.000996,1.969123,1.506852


In [387]:
# 위 출력에 ['camp']으로 analytic CAPM 정보 조회 가능
dict_V = capm_V['capm']
dict_V

{'ticker': 'V',
 'beta': 1.0166235331680278,
 'r_CAPM': 0.0012053763663069789,
 'cov': 0.0008042703452127653,
 'r_M': 0.001968913309352517,
 'r_f': 0.0479}

# <span style="color:tomato">(주의) 이하의 차트 출력 결과를 나타내려면 plotly 라이브러리 설치가 필요합니다 (단순 이미지가 아닌 상호작용 가능 차트)<span>

In [377]:
# !pip install plotly

In [374]:
showCandleStickChart(df_V, 'V')

In [375]:
showPriceSPXPlot(df_V, 'V')

In [376]:
showLinREgCAPMPlot(df_V, 'V')

V: CAPM beta = 1.0166


# (실행 예시) S&P500 기업들 목록에서 무작위 5개 기업을 추출해 순식간에 CAPM 결과를 출력하고 다양한 차트를 그려내기

## 1. S&P500 기업 중 5개 기업을 무작위 추출

In [378]:
import random 
tickers = random.sample(list(sp500['Ticker']), 5)
tickers

['AMT', 'HST', 'AEE', 'DIS', 'WELL']

## 2. 반복문으로 자동화 함수 operateCAPM 반복실행, 결과 저장

In [388]:
result_bucket = []
for ticker in tickers:
    capm_result = operateCAPM(ticker)
    result_bucket.append(capm_result)

## 3. 첫번째 기업에 대해 데이터프레임과 계산 결과를 조회

In [389]:
result_bucket[0]['df']

,timestamp,open,high,low,close,volume,SPX,return_price,return_SPX,normed_price,normed_SPX
0,2018-01-05,143.16,143.22,139.58,140.51,7473333,2743.15,-0.015140,0.026010,1.000000,1.000000
1,2018-01-12,140.66,143.24,132.54,133.00,13118389,2786.24,-0.053448,0.015708,0.946552,1.015708
2,2018-01-19,133.24,138.00,133.24,137.62,11578777,2810.30,0.034737,0.008635,0.979432,1.024479
3,2018-01-26,137.69,145.14,137.41,145.13,11794075,2872.87,0.054571,0.022264,1.032880,1.047289
4,2018-02-02,144.90,149.36,142.84,145.22,14269541,2762.13,0.000620,-0.038547,1.033521,1.006919
...,...,...,...,...,...,...,...,...,...,...,...
273,2023-03-31,200.19,204.49,190.96,204.34,9690176,4109.31,0.023645,0.034833,1.454274,1.498026
274,2023-04-06,203.19,207.76,199.12,206.69,9641366,4105.02,0.011500,-0.001044,1.470999,1.496462
275,2023-04-14,205.13,213.48,203.77,206.89,9011035,4137.64,0.000968,0.007946,1.472422,1.508354
276,2023-04-21,207.77,211.94,202.83,204.14,8809086,4133.52,-0.013292,-0.000996,1.452850,1.506852


## $ E(r_i) = r_f + \beta_i(r_M - r_f) $

In [390]:
result_bucket[0]['capm']

{'ticker': 'AMT',
 'beta': 0.8395781367695004,
 'r_CAPM': 0.009279281281915082,
 'cov': 0.0006654971805951778,
 'r_M': 0.0018998517985611505,
 'r_f': 0.0479}

## 4. 여러 기업들에 대해 다양한 시각화 플롯들을 한꺼번에 출력해보기

In [382]:
for i in range(len(tickers)):
    showCandleStickChart(result_bucket[i]['df'], tickers[i])
    showPriceSPXPlot(result_bucket[i]['df'], tickers[i])    
    showLinREgCAPMPlot(result_bucket[i]['df'], tickers[i])        

AMT: CAPM beta = 0.8396


HST: CAPM beta = 1.1841


AEE: CAPM beta = 0.6999


DIS: CAPM beta = 1.0723


WELL: CAPM beta = 1.2239
